In [ ]:
'''
softmax지나면 확률값나온다 여러번 시행해서 가장 많이 나온것을 택하거나 확률이 가장높은것을
택하거나 확률이 보이게 해보자 softmax로 '''

In [256]:
import math
import random

def generate_data(num_trials):
    trials = []
    for _ in range(num_trials):
        count = 0
        while True:
            count += 1
            if random.randint(1, 125) == 1:
                break
        trials.append(count)
    return trials

x_data = generate_data(1000)

#print("X 데이터:", x_data)

In [257]:
def assign_numbers_based_on_range(numbers, ranges):
    new_list = []
    for number in numbers:
        assigned = False
        for range_start, range_end, assigned_number in ranges:
            if range_start <= number <= range_end:
                new_list.append(assigned_number)
                assigned = True
                break
        if not assigned:
            new_list.append(ranges[-1][2])  # 범위에 속하지 않는 경우 마지막 숫자를 할당(range의 마지막 요소 세번째 요소)
    return new_list

# 사용 예시
numbers = x_data
ranges = [(1, 51, 1), (51, 101, 2), (101, 151, 3), (151, 201, 4),(201, 251, 5),(251, 301, 6)] # 각 세번째 요소는 할당할 숫자를 의미합니다.
normalization_x_data = assign_numbers_based_on_range(numbers, ranges)
#print("normalization_x_data =", normalization_x_data)

In [258]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split

# 데이터 예시 (리스트 형태)
data = normalization_x_data

# 시퀀스 데이터 생성
sequence_length = 3  # 시퀀스 길이 설정
generator = TimeseriesGenerator(data, data, length=sequence_length, batch_size=1)

# 데이터 분할
X, y = [], []
for i in range(len(generator)):
    x, y_ = generator[i]
    X.append(x[0])
    y.append(y_[0])

X = np.array(X)
y = np.array(y)

# 훈련, 검증, 테스트 세트 분할
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train Set: {X_train.shape}, Validation Set: {X_val.shape}, Test Set: {X_test.shape}")
print(X)

Train Set: (697, 3), Validation Set: (150, 3), Test Set: (150, 3)
[[4 6 2]
 [6 2 2]
 [2 2 1]
 ...
 [4 2 1]
 [2 1 1]
 [1 1 3]]


In [259]:
'''
##before
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# 데이터셋 생성 함수
def create_dataset(numbers, n_steps):
    X, y = [], []
    for i in range(len(numbers)):
        end_ix = i + n_steps
        if end_ix > len(numbers)-1:
            break
        seq_x, seq_y = numbers[i:end_ix], numbers[end_ix] # end_ix는 포함되지 않는다.
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# 숫자 시퀀스 정의
numbers = normalization_x_data
n_steps = 2 # 학습하고싶은 숫자묶음의 크기(발걸음)을 설정합니다.

# 데이터셋 생성
X, y = create_dataset(numbers, n_steps) #데이터셋 생성 함수 활성화

print(X)
print(y)
'''

'\n##before\nfrom keras.models import Sequential\nfrom keras.layers import LSTM, Dense\nimport numpy as np\n\n# 데이터셋 생성 함수\ndef create_dataset(numbers, n_steps):\n    X, y = [], []\n    for i in range(len(numbers)):\n        end_ix = i + n_steps\n        if end_ix > len(numbers)-1:\n            break\n        seq_x, seq_y = numbers[i:end_ix], numbers[end_ix] # end_ix는 포함되지 않는다.\n        X.append(seq_x)\n        y.append(seq_y)\n    return np.array(X), np.array(y)\n\n# 숫자 시퀀스 정의\nnumbers = normalization_x_data\nn_steps = 2 # 학습하고싶은 숫자묶음의 크기(발걸음)을 설정합니다.\n\n# 데이터셋 생성\nX, y = create_dataset(numbers, n_steps) #데이터셋 생성 함수 활성화\n\nprint(X)\nprint(y)\n'

In [260]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import MeanAbsoluteError

# 모델 정의
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# 모델 훈련
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

# 모델 성능 평가
test_loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

Epoch 1/100
22/22 [==============================] - 1s 7ms/step - loss: 8.8993 - val_loss: 9.3072
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 6.6602 - val_loss: 6.4163
Epoch 3/100
22/22 [==============================] - 0s 2ms/step - loss: 4.5923 - val_loss: 4.2657
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 4.2283 - val_loss: 4.3800
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 4.0154 - val_loss: 4.0939
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 3.8159 - val_loss: 3.9086
Epoch 7/100
22/22 [==============================] - 0s 2ms/step - loss: 3.6699 - val_loss: 3.8706
Epoch 8/100
22/22 [==============================] - 0s 2ms/step - loss: 3.5194 - val_loss: 3.5707
Epoch 9/100
22/22 [==============================] - 0s 2ms/step - loss: 3.3828 - val_loss: 3.5199
Epoch 10/100
22/22 [==============================] - 0s 2ms/step - loss: 3.2884 - val_loss: 3.4306
Epoch 11/

In [224]:
'''
##before
# LSTM 모델 정의
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# 모델 훈련을 위해 X를 재구성
X = X.reshape((X.shape[0], X.shape[1], 1))

# 조기 종료 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 훈련 (내가 생각했을떄 이것은 같은 상황의 반복이므로 에포크를 높게할수록 잘적중될것이다. 그러나 학습하더라도 램덤떄문에 성능개선에 한계가 있다)
model.fit(X, y, epochs=20, verbose=1, callbacks=[early_stopping])
'''

"\n##before\n# LSTM 모델 정의\nfrom keras.models import Sequential\nfrom keras.layers import LSTM, Dense\nfrom keras.callbacks import EarlyStopping\n\nmodel = Sequential()\nmodel.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))\nmodel.add(Dense(1))\nmodel.compile(optimizer='adam', loss='mse')\n\n# 모델 훈련을 위해 X를 재구성\nX = X.reshape((X.shape[0], X.shape[1], 1))\n\n# 조기 종료 콜백 설정\nearly_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)\n\n# 모델 훈련 (내가 생각했을떄 이것은 같은 상황의 반복이므로 에포크를 높게할수록 잘적중될것이다. 그러나 학습하더라도 램덤떄문에 성능개선에 한계가 있다)\nmodel.fit(X, y, epochs=20, verbose=1, callbacks=[early_stopping])\n"

In [265]:
def assign_numbers_based_on_range(numbers, ranges):
    new_list = []
    for number in numbers:
        assigned = False
        for range_start, range_end, assigned_number in ranges:
            if range_start <= number <= range_end:
                new_list.append(assigned_number)
                assigned = True
                break
        if not assigned:
            new_list.append(ranges[-1][2])  # 범위에 속하지 않는 경우 마지막 숫자를 할당(range의 마지막 요소 세번째 요소)
    return new_list

# 사용 예시
numbers = [209, 109, 68]
ranges = [(1, 51, 1), (51, 101, 2), (101, 151, 3), (151, 201, 4),(201, 251, 5),(251, 301, 6)] # 각 세번째 요소는 할당할 숫자를 의미합니다.
normalization_input = assign_numbers_based_on_range(numbers, ranges)
print("normalization_input =", normalization_input)

## 여기까지가 새로들어온 input값 구간으로 정규화시켜주는 부분

normalization_input = [5, 3, 2]


In [266]:
new_sequence = np.array([5, 3, 2])  # 새 시퀀스 값들
new_sequence = new_sequence.reshape((1, sequence_length, 1))  # 모델에 맞게 형태 변환

# 예측 수행
predicted_value = model.predict(new_sequence)

# 예측 결과 출력
print(f"Predicted Value: {predicted_value[0][0]}")

# 예측의 불확실성 추정 (간단한 예시)
# 이 부분은 실제 확률을 나타내는 것이 아니라, 단순히 오차를 기반으로 한 추정입니다.
error_margin = 0.1  # 가정된 오차 범위
lower_bound = predicted_value[0][0] - error_margin
upper_bound = predicted_value[0][0] + error_margin
print(f"Predicted Value with Error Margin: {lower_bound} to {upper_bound}")

1/1 [==============================] - 0s 8ms/step
Predicted Value: 2.633702278137207
Predicted Value with Error Margin: 2.533702278137207 to 2.733702278137207


# 내가 만든 모델 성능 알수있도록 해주는 방법 만들기(확률을 나타내든 정확도를 나타내든)